In [1]:
import pandas as pd
import numpy as np

store_data = pd.read_csv("./data/store.csv")
store_data

train_data = pd.read_csv("./data/train.csv")
train_data

/Users/corstiaen/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,2013-01-01,1115.0,2.0,0.0,0.0,0.0,0.0,a,1.0
1,2013-01-01,379.0,2.0,0.0,0.0,0.0,0.0,a,1.0
2,2013-01-01,378.0,2.0,0.0,0.0,0.0,0.0,a,1.0
3,2013-01-01,377.0,2.0,0.0,0.0,0.0,0.0,a,1.0
4,2013-01-01,376.0,2.0,0.0,0.0,0.0,0.0,a,1.0
...,...,...,...,...,...,...,...,...,...
637769,2014-07-31,752.0,4.0,7259.0,576.0,1.0,1.0,0,1.0
637770,2014-07-31,753.0,NaN,9938.0,796.0,1.0,1.0,0,1.0
637771,2014-07-31,754.0,4.0,10564.0,1162.0,1.0,1.0,0,NaN
637772,2014-07-31,755.0,4.0,12302.0,1020.0,1.0,1.0,0,1.0


In [2]:
store_data

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [3]:
train_data.drop("Customers", axis=1, inplace=True)

In [4]:
train_data.dtypes

Date              object
Store            float64
DayOfWeek        float64
Sales            float64
Open             float64
Promo            float64
StateHoliday      object
SchoolHoliday    float64
dtype: object

In [5]:
store_data.dtypes

Store                          int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
dtype: object

In [6]:
# There's NaNs in the store ID

store_data.loc[:, "Store"].min()

1

In [7]:
train_data = train_data.fillna({"Store": 0})

In [8]:
train_data = train_data.astype({"Store": int})

In [9]:
merged_data = train_data.merge(store_data, how="left", on="Store")

In [10]:
merged_data

,Date,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,2013-01-01,1115,2.0,0.0,0.0,0.0,a,1.0,d,c,5350.0,NaN,NaN,1.0,22.0,2012.0,"Mar,Jun,Sept,Dec"
1,2013-01-01,379,2.0,0.0,0.0,0.0,a,1.0,d,a,6630.0,NaN,NaN,0.0,NaN,NaN,NaN
2,2013-01-01,378,2.0,0.0,0.0,0.0,a,1.0,a,c,2140.0,8.0,2012.0,0.0,NaN,NaN,NaN
3,2013-01-01,377,2.0,0.0,0.0,0.0,a,1.0,a,c,100.0,6.0,2010.0,1.0,18.0,2010.0,"Feb,May,Aug,Nov"
4,2013-01-01,376,2.0,0.0,0.0,0.0,a,1.0,a,a,160.0,8.0,2012.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637769,2014-07-31,752,4.0,7259.0,1.0,1.0,0,1.0,a,a,970.0,3.0,2013.0,1.0,31.0,2013.0,"Feb,May,Aug,Nov"
637770,2014-07-31,753,NaN,9938.0,1.0,1.0,0,1.0,d,c,540.0,11.0,2012.0,1.0,35.0,2010.0,"Mar,Jun,Sept,Dec"
637771,2014-07-31,754,4.0,10564.0,1.0,1.0,0,NaN,c,c,380.0,5.0,2008.0,1.0,10.0,2014.0,"Mar,Jun,Sept,Dec"
637772,2014-07-31,755,4.0,12302.0,1.0,1.0,0,1.0,d,c,13130.0,12.0,2003.0,0.0,NaN,NaN,NaN


In [17]:
from sklearn.model_selection import train_test_split

X = merged_data.loc[:, ~merged_data.columns.isin(["Sales"])]
y = merged_data.loc[:, "Sales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=.8)

In [18]:
X_train.head()

,Date,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
143027,2013-05-09,83,4.0,0.0,NaN,a,0.0,a,a,2710.0,NaN,NaN,0.0,NaN,NaN,NaN
553420,2014-05-12,68,1.0,NaN,0.0,0,0.0,a,c,250.0,NaN,NaN,1.0,35.0,2012.0,"Mar,Jun,Sept,Dec"
589010,2014-06-13,88,5.0,1.0,0.0,0,0.0,a,a,10690.0,10.0,2005.0,0.0,NaN,NaN,NaN
227227,2013-07-23,685,2.0,1.0,0.0,0,1.0,a,a,650.0,11.0,2013.0,1.0,37.0,2009.0,"Jan,Apr,Jul,Oct"
561723,2014-05-19,689,1.0,1.0,1.0,0,0.0,d,a,15040.0,10.0,2004.0,0.0,NaN,NaN,NaN


In [19]:
len(X_train)

510219

In [20]:
len(X_test)

127555

In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(X_train)
profile.to_file(output_file="rossman_profile_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/corstiaen/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:4812: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))
